In [25]:
#host - trusted-prediction-research.c8qtxwbvasgw.ap-south-1.rds.amazonaws.com
#user - postgres
#passoword - ynKrco0QWa56iez8sKtG

In [1]:
#train data from date1 to date2
#date1 = 1
#year1 = 2014
#month1 = 1
#date2 = 20
#month2 = 11
#year2 = 2020
# Testing on match
#test_date = 1
#test_month = 11
#test_year = 2020
#start_time = 8  #for afternoon match
#start_time = 12 #for evening match

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
host_="trusted-prediction-research.c8qtxwbvasgw.ap-south-1.rds.amazonaws.com"
user_= "postgres"
passw= "ynKrco0QWa56iez8sKtG"

In [5]:
import numpy as np
import pandas as pd
import psycopg2

# Extracting IPL Data

In [6]:
conn = psycopg2.connect("dbname=cricket host =sportsanalytics.in user=postgres password=Sumit@123 ")

In [7]:
series = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4", conn)

In [8]:
query = f"""SELECT id,start_date FROM cds_match_match WHERE series_id in {tuple(series['id'])} AND start_date BETWEEN '{year1}-{month1}-{date1} 00:00:01' AND '{year2}-{month2}-{date2} 23:59:59' """ 
match = pd.read_sql_query(query, conn)
query = f"""SELECT id,match_id,team_id FROM cds_scorecard_scorecard WHERE match_id in {tuple(match['id'])}"""
scorecard_scorecard = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_scorecard_matchplayers WHERE match_id in {tuple(match['id'])} """
player_data = pd.read_sql_query(query,conn)
player_data = player_data[{'isAllrounder','isBatsmen','isBowler','isWicketKeeper','player_id'}]

In [9]:
query = f"""SELECT * FROM cds_match_inning_matchbatsmaninning WHERE scorecard_id in {tuple(scorecard_scorecard['id'])}"""
batsman_data = pd.read_sql_query(query,conn)

In [10]:
query = f"""SELECT * FROM cds_match_inning_matchbowlersinning WHERE scorecard_id in {tuple(scorecard_scorecard['id'])}"""
bowler_data = pd.read_sql_query(query,conn)

In [11]:
batsman_inning = pd.read_sql_query("SELECT * FROM cds_match_inning_matchbatsmaninning", conn)
bowl_inning = pd.read_sql_query("SELECT * FROM cds_match_inning_matchbowlersinning", conn)
batsman_data = batsman_inning[batsman_inning['scorecard_id'].isin(scorecard_scorecard['id'])] 
bowler_data = bowl_inning[bowl_inning['scorecard_id'].isin(scorecard_scorecard['id'])]

In [12]:
query = f"""SELECT id,start_date FROM cds_match_match WHERE series_id in {tuple(series['id'])} AND start_date BETWEEN '2020-01-01 00:00:01' AND '2020-11-04 23:59:59' """ 
match2020 = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_scorecard_matchplayers WHERE match_id in {tuple(match2020['id'])} """
player_data2020 = pd.read_sql_query(query,conn)
playerid_team2020 = dict(zip(player_data2020['player_id'],player_data2020['team_id']))

In [13]:
playerid_name = pd.read_sql_query("SELECT id,name FROM cds_player_player", conn)
playerid_name = dict(zip(playerid_name['id'],playerid_name['name']))
teamid_name = pd.read_sql_query("SELECT id,name FROM cds_team_team", conn)
teamid_name = dict(zip(teamid_name['id'],teamid_name['name']))

# Data preprocessing and Fantasy points calculation

In [14]:
batsman_data = batsman_data[{'balls','runs','strike_rate','fours','sixes','player_id','scorecard_id'}]
bowler_data = bowler_data[{'economy','wickets','player_id','scorecard_id'}]

In [15]:
batting_data = batsman_data.merge(player_data, how='left', on='player_id').drop_duplicates()
bowling_data = bowler_data.merge(player_data,how='left', on='player_id').drop_duplicates()

In [16]:
bowling_data['fantasy_points'] = (25*bowling_data['wickets'] +2*(bowling_data['economy']<4)+2*(bowling_data['economy']<5) +2*(bowling_data['economy']<6) -2*(bowling_data['economy']>9)-2*(bowling_data['economy']>10) - 2*(bowling_data['economy']>11)+ 8*(bowling_data['wickets']>=4)+8*(bowling_data['wickets']>=5))
batting_data['fantasy_points'] = batting_data['runs'] + (batting_data['fours'] + batting_data['sixes']) + 2*batting_data['sixes'] + 8*(batting_data['runs']>=50) + 8*(batting_data['runs']>=100) - 2*(batting_data['runs']==0) - 2*(batting_data['balls']>=10)*(batting_data['strike_rate']<70)*(batting_data['strike_rate']>=60) - 4*(batting_data['balls']>=10)*(batting_data['strike_rate']<60)*(batting_data['strike_rate']>=50) - 6*(batting_data['balls']>=10)*(batting_data['strike_rate']<50)

In [17]:
batting_data = batting_data[{'scorecard_id','fantasy_points','player_id'}]  
bowling_data = bowling_data[{'scorecard_id','fantasy_points','player_id'}]

In [18]:
scorecard_scorecard['scorecard_id'] = scorecard_scorecard['id']
teams = np.unique(np.array(list(scorecard_scorecard['team_id'])))
scorecard_scorecard = scorecard_scorecard[{'match_id','scorecard_id'}]

In [19]:
bowling_data = bowling_data.merge(scorecard_scorecard,how='left', on='scorecard_id')
batting_data = batting_data.merge(scorecard_scorecard,how='left', on='scorecard_id')
players_data = pd.concat([batting_data, bowling_data],0)
players_data = players_data[{'player_id','fantasy_points','match_id'}]

In [20]:
grouped_data = players_data.groupby(['player_id','match_id']).sum()

In [21]:
wicketkeeper = dict(zip(player_data['player_id'],player_data['isWicketKeeper'].astype(int)))
bowlers = dict(zip(player_data['player_id'],player_data['isBowler'].astype(int)))
batsmen = dict(zip(player_data['player_id'],player_data['isBatsmen'].astype(int)))
allrounder = dict(zip(player_data['player_id'],player_data['isAllrounder'].astype(int)))

In [22]:
#print("All IPL team ids are :")
#for i in teams:
#    print("Id",i,teamid_name[i])

All IPL team ids are :
Id 20 Chennai Super Kings
Id 21 Mumbai Indians
Id 22 Delhi Capitals
Id 23 Sunrisers Hyderabad
Id 24 Kolkata Knight Riders
Id 25 Kings XI Punjab
Id 26 Royal Challengers Bangalore
Id 27 Rajasthan Royals
Id 28 Rising Pune Supergiant
Id 29 Gujarat Lions


# Modelling


## For Eliminator the two teams are 

In [23]:
team1 = 23
team2 = 26

In [24]:
team2

26

## Dream Team code

In [83]:
series_dt = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4", conn)
query = f"""SELECT id,start_date FROM cds_match_match WHERE series_id in {tuple(series_dt['id'])} AND start_date BETWEEN '{test_year}-{test_month}-{test_date} {start_time}:00:01' AND '{test_year}-{test_month}-{test_date} 23:59:59' """ 
match_dt = pd.read_sql_query(query, conn)
mid = match_dt['id'][0]
query = f"""SELECT id,match_id,team_id FROM cds_scorecard_scorecard WHERE match_id = {mid}"""
scorecard_scorecard_dt = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_match_inning_matchbatsmaninning WHERE scorecard_id in {tuple(scorecard_scorecard_dt['id'])}"""
batsman_inning_dt = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_match_inning_matchbowlersinning WHERE scorecard_id in {tuple(scorecard_scorecard_dt['id'])}"""
bowl_inning_dt = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_match_inning_matchfieldinginning WHERE scorecard_id in {tuple(scorecard_scorecard_dt['id'])}"""
fielding_inning = pd.read_sql_query(query, conn)

In [87]:

query = f"""SELECT * FROM cds_scorecard_matchplayers WHERE match_id = {int(match_dt['id'])} """
player_data_dt = pd.read_sql_query(query,conn)
player_data_dt = player_data_dt[{'isAllrounder','isBatsmen','isBowler','isWicketKeeper','player_id'}]
team1 = np.unique(scorecard_scorecard_dt['team_id'])[0]
team2 = np.unique(scorecard_scorecard_dt['team_id'])[1]
batsman_data_dt = batsman_inning_dt[batsman_inning_dt['scorecard_id'].isin(scorecard_scorecard_dt['id'])] 
bowler_data_dt = bowl_inning_dt[bowl_inning_dt['scorecard_id'].isin(scorecard_scorecard_dt['id'])]
query = f"""SELECT id,start_date FROM cds_match_match WHERE series_id in {tuple(series_dt['id'])} AND start_date BETWEEN '{test_year}-{test_month}-{test_date} {start_time}:00:01' AND '{test_year}-{test_month}-{test_date} 23:59:59' """ 
match2020_dt = pd.read_sql_query(query, conn)
mid2020 = match2020_dt['id'][0]
query = f"""SELECT * FROM cds_scorecard_matchplayers WHERE match_id = {mid2020} """
player_data2020_dt = pd.read_sql_query(query,conn)
playerid_team2020_dt = dict(zip(player_data2020_dt['player_id'],player_data2020_dt['team_id']))
playerid_name_dt = pd.read_sql_query("SELECT id,name FROM cds_player_player", conn)
playerid_name_dt = dict(zip(playerid_name_dt['id'],playerid_name_dt['name']))
#teamid_name = pd.read_sql_query("SELECT id,name FROM cds_team_team", conn)
#teamid_name = dict(zip(teamid_name['id'],teamid_name['name']))
batsman_data_dt = batsman_data_dt[{'balls','runs','strike_rate','fours','sixes','player_id','scorecard_id'}]
bowler_data_dt = bowler_data_dt[{'economy','wickets','player_id','scorecard_id'}]
batting_data_dt = batsman_data_dt.merge(player_data_dt, how='left', on='player_id').drop_duplicates()
bowling_data_dt = bowler_data_dt.merge(player_data_dt,how='left', on='player_id').drop_duplicates()
bowling_data_dt['fantasy_points'] = (25*bowling_data_dt['wickets'] +2*(bowling_data_dt['economy']<4)+2*(bowling_data_dt['economy']<5) +2*(bowling_data_dt['economy']<6) -2*(bowling_data_dt['economy']>9)-2*(bowling_data_dt['economy']>10) - 2*(bowling_data_dt['economy']>11)+ 8*(bowling_data_dt['wickets']>=4)+8*(bowling_data_dt['wickets']>=5))
batting_data_dt['fantasy_points'] = batting_data_dt['runs'] + (batting_data_dt['fours'] + batting_data_dt['sixes']) + 2*batting_data_dt['sixes'] + 8*(batting_data_dt['runs']>=50) + 8*(batting_data_dt['runs']>=100) - 2*(batting_data_dt['runs']==0) - 2*(batting_data_dt['balls']>=10)*(batting_data_dt['strike_rate']<70)*(batting_data_dt['strike_rate']>=60) - 4*(batting_data_dt['balls']>=10)*(batting_data_dt['strike_rate']<60)*(batting_data_dt['strike_rate']>=50) - 6*(batting_data_dt['balls']>=10)*(batting_data_dt['strike_rate']<50)
fielding_points = {}
for i in fielding_inning.index:
    if(list(fielding_points.keys()).count(fielding_inning['fielder_id'][i])==1):
        if(fielding_inning['isCaught'][i]):
            fielding_points[fielding_inning['fielder_id'][i]]+=8
        if(fielding_inning['isRunOut'][i]):
            fielding_points[fielding_inning['fielder_id'][i]]+=12
    else:
        if(fielding_inning['isCaught'][i]):
            fielding_points[fielding_inning['fielder_id'][i]] = 8
        if(fielding_inning['isRunOut'][i]):
            fielding_points[fielding_inning['fielder_id'][i]] = 12
bowling_data_dt = bowling_data_dt[{'scorecard_id','fantasy_points','player_id'}]
batting_data_dt = batting_data_dt[{'scorecard_id','fantasy_points','player_id'}]  
scorecard_scorecard_dt['scorecard_id'] = scorecard_scorecard_dt['id']
teams_dt = np.unique(np.array(list(scorecard_scorecard_dt['team_id'])))
scorecard_scorecard_dt = scorecard_scorecard_dt[{'match_id','scorecard_id'}]
bowling_data_dt = bowling_data_dt.merge(scorecard_scorecard_dt,how='left', on='scorecard_id')
batting_data_dt = batting_data_dt.merge(scorecard_scorecard_dt,how='left', on='scorecard_id')
players_data_dt = pd.concat([batting_data_dt, bowling_data_dt],0)
players_data_dt = players_data_dt[{'player_id','fantasy_points','match_id'}]
grouped_data_dt = players_data_dt.groupby(['player_id','match_id']).sum()
wicketkeeper_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isWicketKeeper'].astype(int)))
bowlers_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isBowler'].astype(int)))
batsmen_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isBatsmen'].astype(int)))
allrounder_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isAllrounder'].astype(int)))
player_team1_team2_dt = list(player_data_dt['player_id'])
fantasy_points_dt = {}
for i in player_team1_team2_dt:
    fantasy_points_dt[i]=0
j = 0
for i in grouped_data_dt.index:
    a = int(i[0])
    fantasy_points_dt[a] = fantasy_points_dt[a] + list(grouped_data_dt['fantasy_points'])[j]
    if(list(fielding_points.keys()).count(a)==1):
        fantasy_points_dt[a] = fantasy_points_dt[a]+fielding_points[a]
    j = j+1
#Sinlgle team prediction code (Mainly for dream team)
from pulp import *
prob = LpProblem("Dream_Team",LpMaximize)
fantasy_team1 = dict({i: LpVariable(cat=LpBinary,name="PLAYER1_{}".format(i)) for i in player_team1_team2_dt}) 
prob+= lpSum([ i for i in fantasy_team1.values()])== 11
objective = lpSum([float(fantasy_points_dt[i]) * fantasy_team1[i] for i in fantasy_team1])
prob += lpSum([fantasy_team1[i]*wicketkeeper_dt[i] for i in player_team1_team2_dt]) >= 1
prob += lpSum([fantasy_team1[i]*batsmen_dt[i] for i in player_team1_team2_dt]) >=3
prob += lpSum([fantasy_team1[i]*bowlers_dt[i] for i in player_team1_team2_dt]) >= 3
prob += lpSum([fantasy_team1[i]*allrounder_dt[i] for i in player_team1_team2_dt]) >=1
prob += objective
prob.solve()
print("Dream Team for match between",teamid_name[team1],"and", teamid_name[team2],"are ")
max_score = 0
for j in range(1,2):
    #print("Team {} is".format(j))
    for i in fantasy_team1:
        if(value(fantasy_team1[i])==1):
            print("Id",i,playerid_name[i],"having points",fantasy_points_dt[i])
            max_score = max_score+fantasy_points_dt[i]
    print(" ")      
print("Score for Dream Team is",max_score)

Dream Team for match between Kolkata Knight Riders and Rajasthan Royals are 
Team 1 is
Id 1031 Shivam Mavi having points 64
Id 1028 Varun Chakravarthy having points 52
Id 33 Shubman Gill having points 42
Id 1017 Rahul Tripathi having points 49
Id 439 Eoin Morgan having points 99
Id 2 Pat Cummins having points 134
Id 154 Jos Buttler having points 50
Id 1020 Rahul Tewatia having points 111
Id 1001 Shreyas Gopal having points 44
Id 604 Jofra Archer having points 35
Id 1067 Kartik Tyagi having points 52
 
Score for Dream Team is 732


## Fantasy points prediction

In [89]:
# All players from LineUps are taken below
player_team1_team2 = player_team1_team2_dt

In [90]:
fantasy_points_pred = {}

In [91]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#player_ids = np.unique(np.concatenate((pd.unique(batting_data['player_id']),pd.unique(bowling_data['player_id']))))
rem = []
for player in player_team1_team2 :
    df = pd.DataFrame(np.array(grouped_data.iloc[grouped_data.index.get_level_values('player_id') == player]['fantasy_points']),columns = ['fantasy_score'])
    if(len(df['fantasy_score'])==1):
        fantasy_points_pred[player] = int(df['fantasy_score'])
        continue
    if(len(df['fantasy_score'])<1):
        rem.append(player)
        continue
    model = SimpleExpSmoothing(np.asarray(df['fantasy_score'])).fit()
    fantasy_points_pred[player] = int(model.forecast())
    del model
    del df
for player in rem:
    player_team1_team2.remove(player)

## Fantasy Team prediction

In [92]:
from pulp import *
predicted_teams = []
no_teams = 10

In [93]:
import operator
sorted_d = sorted(fantasy_points_pred.items(), key=operator.itemgetter(1))
min_3 = []
min_3.append(sorted_d[0][0])
min_3.append(sorted_d[1][0])
min_3.append(sorted_d[2][0])
replace = (min_3[np.random.randint(0,len(min_3))])
player_team1_team2.remove(replace)
for n_teams in range(0,no_teams):
    prob = LpProblem("Dream_Team",LpMaximize)
    fantasy_team1 = dict({i: LpVariable(cat=LpBinary,name="PLAYER1_{}".format(i)) for i in player_team1_team2}) 
    prob+= lpSum([ i for i in fantasy_team1.values()])== 11
    objective = lpSum([float(fantasy_points_pred[i]) * fantasy_team1[i] for i in fantasy_team1]) 
    # each playertype constraint
    prob += lpSum([fantasy_team1[i]*wicketkeeper[i] for i in player_team1_team2]) >= 1
    prob += lpSum([fantasy_team1[i]*batsmen[i] for i in player_team1_team2]) >=3
    prob += lpSum([fantasy_team1[i]*bowlers[i] for i in player_team1_team2]) >= 3
    prob += lpSum([fantasy_team1[i]*allrounder[i] for i in player_team1_team2]) >=1
    prob += objective
    prob.solve()
    pred_teamid = []
    #print("Fantasy teams for match between",teamid_name[team1],"and", teamid_name[team2],"are ")
    for j in range(1,2):
        #print("Team {} is".format(j))
        for i in fantasy_team1:
            #pred_teamid[i] = value(fantasy_team1[i])
            if(value(fantasy_team1[i])==1):
                #print("Id",i,playerid_name[i],"predicted points",fantasy_points_pred[i])
                pred_teamid.append(i)
    predicted_teams.append(pred_teamid)
    pred_teamid.remove(pred_teamid[np.random.randint(0,len(pred_teamid))])
    player_team1_team2.append(replace)
    #sorted_d = sorted(fantasy_points_pred.items(), key=operator.itemgetter(1))
    min_3 = []
    if(sorted_d[0][0]!=replace): min_3.append(sorted_d[0][0])
    if(sorted_d[1][0]!=replace): min_3.append(sorted_d[1][0])
    if(sorted_d[2][0]!=replace): min_3.append(sorted_d[2][0])
    if(sorted_d[3][0]!=replace): min_3.append(sorted_d[3][0])
    replace = (min_3[np.random.randint(0,len(min_3))])
    player_team1_team2.remove(replace)
    

In [94]:
predicted_teams
pred_scores = []
teamn = 1
for i in predicted_teams:
    s = 0
    for j in i:
        s = s + fantasy_points_dt[j]
    pred_scores.append(s)
    print(f"our score for team number {teamn} is {s}")
    teamn = teamn + 1

322
316
411
365
416
475
400
360
469
392
